In [ ]:
#hide
from nbdev import *
%nbdev_default_export test

Cells will be exported to nbdev.test,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
#export
from nbdev.imports import *
from nbdev.sync import *
from nbdev.export import *
from nbdev.export import _mk_flag_re
from nbdev.export2html import _re_notebook2script

from nbconvert.preprocessors import ExecutePreprocessor

# Extract tests

> The functions that grab the cells containing tests (filtering with potential flags) and execute them

Everything that is not an exported cell is considered a test, so you should make sure your notebooks can all run smoothly (and fast) if you want to use this functionality as the CLI. You can mark some cells with special flags (like slow) to make sure they are only executed when you authorize it. Those flags should be configured in your `settings.ini` (separated by a `|` if you have several of them). You can also apply flags to one entire notebook by using the `all` option of the test flag, e.g. `%nbdev_slow_test all`, in code cells.

If `tst_flags=slow|fastai` in `settings.ini`, you can:
- mark slow tests with the `%nbdev_slow_test` flag
- mark tests that depend on fastai with the `%nbdev_fastai_test` flag.

## Detect flags

The following functions detect the cells that should be excluded from the tests (unless their special flag is passed).

In [ ]:
#export
_re_all_flag = ReTstFlags(True)

In [ ]:
#export
def get_all_flags(cells):
    "Check for all test flags in `cells`"
    if len(Config().get('tst_flags',''))==0: return []
    result = []
    for cell in cells:
        if cell['cell_type'] == 'code': result.extend(_re_all_flag.findall(cell['source']))
    return set(result)

In [ ]:
nb = read_nb("04_test.ipynb")
assert get_all_flags(nb['cells']) == set()

In [ ]:
#hide
tst_flags_bck=Config().get('tst_flags')
try:
    Config()['tst_flags'] = 'fastai|vslow'
    if hasattr(_re_all_flag, '_re'): del _re_all_flag._re
    cells = [{'cell_type': cell_type, 'source': source} for cell_type, source in [
        ('code', '# export\nfrom local.core import *'), 
        ('markdown', '# title of some kind'), 
        ('code', '# all_vslow \n# all_fastai'),
        ('code', '%nbdev_vslow_test all\n# all_fastai'),
        ('code', '%nbdev_vslow_test all '),
        ('code', '# all_fastai'),
        ('code', '%nbdev_fastai_test  all\n')]]
    for i in range(3):
        test_eq(set(['vslow','fastai']), get_all_flags(cells))
        cells.pop(2)
    for i in range(2):
        test_eq(set(['fastai']), get_all_flags(cells))
        cells.pop(2)
    test_eq(set(), get_all_flags(cells))
finally:
    Config()['tst_flags'] = tst_flags_bck
    del _re_all_flag._re

In [ ]:
#export
_re_flags = ReTstFlags(False)

In [ ]:
#export
def get_cell_flags(cell):
    "Check for any special test flag in `cell`"
    if cell['cell_type'] != 'code' or len(Config().get('tst_flags',''))==0: return []
    return _re_flags.findall(cell['source'])

In [ ]:
test_eq(get_cell_flags({'cell_type': 'code', 'source': "#hide\n%nbdev_fastai_test\n"}), ['fastai'])
test_eq(get_cell_flags({'cell_type': 'code', 'source': "#hide\n"}), [])

In [ ]:
#hide
for expected, flag in [(['fastai'], 'fastai'), ([], 'vslow')]:
    test_eq(expected, get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n# {flag}\n")))
    test_eq(expected, get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n%nbdev_{flag}_test\n")))
    test_eq(expected, get_cell_flags(nbformat.v4.new_code_cell(f"# {flag}\n#hide\n")))
    test_eq(expected, get_cell_flags(nbformat.v4.new_code_cell(f"%nbdev_{flag}_test\n#hide\n")))
    test_eq([], get_cell_flags(nbformat.v4.new_code_cell("#hide\n")))
    test_eq([], get_cell_flags(nbformat.v4.new_code_cell(f"# all_{flag}")))
    test_eq([], get_cell_flags(nbformat.v4.new_code_cell(f"%nbdev_{flag}_test all")))
tst_flags_bck=Config().get('tst_flags')
try:
    Config()['tst_flags'] = 'fastai|vslow'
    del _re_flags._re
    test_eq(['vslow'], get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n# vslow\n")))
    test_eq(['vslow'], get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n%nbdev_vslow_test\n")))
    test_eq(['vslow', 'fastai'], get_cell_flags(nbformat.v4.new_code_cell(f"#hide\n# vslow\n# fastai")))
    test_eq(['fastai', 'vslow'], get_cell_flags(nbformat.v4.new_code_cell(f"%nbdev_fastai_test\n%nbdev_vslow_test")))
finally:
    Config()['tst_flags'] = tst_flags_bck
    del _re_flags._re

## Testing a notebook

In [ ]:
#export
class NoExportPreprocessor(ExecutePreprocessor):
    "An `ExecutePreprocessor` that executes cells that don't have a flag in `flags`"
    def __init__(self, flags, **kwargs):
        self.flags = flags
        super().__init__(**kwargs)

    def preprocess_cell(self, cell, resources, index):
        if 'source' not in cell or cell['cell_type'] != "code": return cell, resources
        for f in get_cell_flags(cell):
            if f not in self.flags: return cell, resources
        if check_re(cell, _re_notebook2script): return cell, resources
        return super().preprocess_cell(cell, resources, index)

In [ ]:
#export
def test_nb(fn, flags=None):
    "Execute tests in notebook in `fn` with `flags`"
    os.environ["IN_TEST"] = '1'
    if flags is None: flags = []
    try:
        nb = read_nb(fn)
        nb = call_cb('begin_test_nb', nb, fn, flags)
        for f in get_all_flags(nb['cells']):
            if f not in flags: return
        ep = NoExportPreprocessor(flags, timeout=600, kernel_name='python3')
        pnb = nbformat.from_dict(nb)
        ep.preprocess(pnb)
        nb = call_cb('after_test_nb', fn)
    finally: os.environ.pop("IN_TEST")

#hide
nbdev users should not need to `import nbdev_callbacks` but we need to patch `begin_test_nb` and `after_test_nb` on to the `nbdev_callbacks` module to test that `test_nb` is calling the callbacks properly.

In [ ]:
#hide
class TestCallbacks:
    def begin_test_nb(self, nb, file_name, flags):
        self.begin_test_nb_data=dict(nb=nb,file_name=file_name,flags=flags)
        return nb
    def after_test_nb(self, file_name):
        self.after_test_nb_data=dict(file_name=file_name)
test_callbacks=TestCallbacks()
call_cb('This makes sure nbdev_callbacks is loaded from the right place')
import nbdev_callbacks
original_callbacks=nbdev_callbacks.begin_test_nb,nbdev_callbacks.after_test_nb
try:
    nbdev_callbacks.begin_test_nb=test_callbacks.begin_test_nb
    nbdev_callbacks.after_test_nb=test_callbacks.after_test_nb
    assert not hasattr(test_callbacks,'begin_test_nb_data')
    expected_file_name,expected_flags='../tests/single-cell-index.ipynb',['slow','cuda']
    test_nb(expected_file_name,expected_flags)
    assert len(test_callbacks.begin_test_nb_data['nb']['cells']) == 1
    assert test_callbacks.begin_test_nb_data['file_name'] == expected_file_name
    assert test_callbacks.begin_test_nb_data['flags'] == expected_flags
    assert test_callbacks.after_test_nb_data['file_name'] == expected_file_name
finally:
    nbdev_callbacks.begin_test_nb,nbdev_callbacks.after_test_nb=original_callbacks

## Export-

In [ ]:
#hide
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_merge.ipynb.
Converted 05a_conda.ipynb.
Converted 06_cli.ipynb.
Converted 07_clean.ipynb.
Converted 08_flag_tests.ipynb.
Converted 09_nbdev_callback_test.ipynb.
Converted 99_search.ipynb.
Converted index.ipynb.
Converted tutorial.ipynb.
